In [17]:
from faker import Faker
import pandas as pd
import numpy as np
import random
from datetime import timedelta

fake = Faker()

def generate_fx_leads(num_leads=100):
    leads = []

    for _ in range(num_leads):
        lead = {
            'lead_id': fake.uuid4(),
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'company': fake.company(),
            'industry': random.choice([
                'Technology', 'Finance', 'Manufacturing',
                'Retail', 'Healthcare', 'Logistics'
            ]),
            'email': fake.email(),
            'phone': fake.phone_number(),
            'country': fake.country(),

            'website_visits': random.randint(1, 10),
            'time_on_site': round(random.uniform(0.5, 15), 2),
            'pages_viewed': random.randint(1, 7),

            'emails_received': random.randint(1, 5),
            'email_opens': random.randint(0, 5),
            'email_clicks': random.randint(0, 3),

            'linkedin_views': random.randint(0, 10),
            'linkedin_interactions': random.randint(0, 5),

            'potential_deal_size': round(random.uniform(50000, 5000000), 2),
            'lead_score': random.randint(1, 100),
            'conversion_likelihood': round(random.uniform(0, 1), 2)
        }

        base_date = fake.date_between(start_date='-6m', end_date='today')
        lead['first_interaction_date'] = base_date
        lead['last_interaction_date'] = base_date + timedelta(days=random.randint(1, 60))

        leads.append(lead)

    return pd.DataFrame(leads)

fx_leads_df = generate_fx_leads(1000)

fx_leads_df.to_csv('fx_leads_dataset.csv', index=False)

print(fx_leads_df.describe())

high_potential_leads = fx_leads_df[fx_leads_df['lead_score'] > 70]
medium_potential_leads = fx_leads_df[(fx_leads_df['lead_score'] > 40) & (fx_leads_df['lead_score'] <= 70)]
low_potential_leads = fx_leads_df[fx_leads_df['lead_score'] <= 40]

print("\nLead Segmentation:")
print(f"High Potential Leads: {len(high_potential_leads)}")
print(f"Medium Potential Leads: {len(medium_potential_leads)}")
print(f"Low Potential Leads: {len(low_potential_leads)}")

       website_visits  time_on_site  pages_viewed  emails_received  \
count     1000.000000   1000.000000   1000.000000      1000.000000   
mean         5.519000      7.606750      4.039000         2.988000   
std          2.925097      4.276683      1.977724         1.401364   
min          1.000000      0.510000      1.000000         1.000000   
25%          3.000000      4.007500      2.000000         2.000000   
50%          6.000000      7.350000      4.000000         3.000000   
75%          8.000000     11.432500      6.000000         4.000000   
max         10.000000     15.000000      7.000000         5.000000   

       email_opens  email_clicks  linkedin_views  linkedin_interactions  \
count   1000.00000   1000.000000     1000.000000              1000.0000   
mean       2.50700      1.501000        5.039000                 2.5050   
std        1.74327      1.117698        3.163778                 1.6825   
min        0.00000      0.000000        0.000000                 0.00

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio

df = pd.read_csv('fx_leads_dataset.csv')

def prepare_features(df):
    features = [
        'website_visits',
        'time_on_site',
        'pages_viewed',
        'emails_received',
        'email_opens',
        'email_clicks',
        'linkedin_views',
        'potential_deal_size'
    ]

    df['is_high_potential'] = (df['lead_score'] > 70).astype(int)

    X = df[features]
    y = df['is_high_potential']

    return X, y

X, y = prepare_features(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_classifier = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
rf_classifier.fit(X_train_scaled, y_train)

y_pred = rf_classifier.predict(X_test_scaled)
print("Classification Report:")
print(classification_report(y_test, y_pred))

feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_classifier.feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

def create_interactive_prospect_plot(df):
    X_scaled = scaler.transform(X)
    df['prospect_probability'] = rf_classifier.predict_proba(X_scaled)[:, 1]

    fig = px.scatter(
        df,
        x='potential_deal_size',
        y='time_on_site',
        color='prospect_probability',
        hover_data=[
            'company',
            'industry',
            'lead_score',
            'prospect_probability'
        ],
        title='Prospect Potential Visualization',
        labels={
            'potential_deal_size': 'Potential Deal Size',
            'time_on_site': 'Time on Site (minutes)',
            'prospect_probability': 'Prospect Probability'
        },
        color_continuous_scale='viridis'
    )

    fig.update_layout(
        height=600,
        width=1000,
        template='plotly_white'
    )

    pio.write_html(fig, file='prospect_potential_visualization.html')

    return fig

interactive_plot = create_interactive_prospect_plot(df)

def segment_prospects(df):
    df['prospect_segment'] = pd.cut(
        df['prospect_probability'],
        bins=[0, 0.3, 0.7, 1],
        labels=['Low Potential', 'Medium Potential', 'High Potential']
    )

    segment_summary = df.groupby('prospect_segment').agg({
        'potential_deal_size': ['mean', 'count'],
        'lead_score': 'mean'
    })

    print("\nProspect Segmentation Summary:")
    print(segment_summary)

    return df

df_segmented = segment_prospects(df)

df_segmented.to_csv('segmented_prospects.csv', index=False)

def create_summary_dashboard(df):
    industry_deals = df.groupby('industry')['potential_deal_size'].mean().sort_values(ascending=False)

    fig = go.Figure(data=[
        go.Bar(
            x=industry_deals.index,
            y=industry_deals.values,
            text=[f'${x:,.0f}' for x in industry_deals.values],
            textposition='auto'
        )
    ])

    fig.update_layout(
        title='Average Potential Deal Size by Industry',
        xaxis_title='Industry',
        yaxis_title='Average Potential Deal Size'
    )

    pio.write_html(fig, file='industry_deal_size_dashboard.html')

create_summary_dashboard(df)

print("Analysis complete. Check the generated HTML files for interactive visualizations.")

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.93      0.82       147
           1       0.17      0.04      0.06        53

    accuracy                           0.69       200
   macro avg       0.45      0.48      0.44       200
weighted avg       0.58      0.69      0.62       200


Feature Importance:
               feature  importance
7  potential_deal_size    0.207366
1         time_on_site    0.196882
6       linkedin_views    0.131554
0       website_visits    0.114717
2         pages_viewed    0.102339
4          email_opens    0.088468
3      emails_received    0.083999
5         email_clicks    0.074675

Prospect Segmentation Summary:
                 potential_deal_size       lead_score
                                mean count       mean
prospect_segment                                     
Low Potential           2.529593e+06   635  37.502362
Medium Potential        2.613602e+06   160  58.262500
High Potenti

<ipython-input-18-81e36b0aa826>:101: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import plotly.express as px
import plotly.io as pio
import random
from faker import Faker

fake = Faker()

def generate_fx_leads(num_leads=200):
    leads = []

    for _ in range(num_leads):
        lead = {
            'lead_id': fake.uuid4(),
            'company': fake.company(),
            'industry': random.choice([
                'Technology', 'Finance', 'Manufacturing',
                'Retail', 'Healthcare', 'Logistics'
            ]),
            'website_visits': random.randint(1, 10),
            'time_on_site': round(random.uniform(0.5, 15), 2),
            'potential_deal_size': round(random.uniform(50000, 5000000), 2),
            'email_opens': random.randint(0, 5),
            'lead_score': random.randint(1, 100)
        }
        leads.append(lead)

    return pd.DataFrame(leads)

df = generate_fx_leads()

def prepare_features(df):
    features = [
        'website_visits',
        'time_on_site',
        'email_opens'
    ]

    df['is_high_potential'] = (df['lead_score'] > 70).astype(int)

    X = df[features]
    y = df['is_high_potential']

    return X, y

X, y = prepare_features(df)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

rf_classifier = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    class_weight='balanced'
)
rf_classifier.fit(X_train_scaled, y_train)

X_scaled = scaler.transform(X)
df['prospect_probability'] = rf_classifier.predict_proba(X_scaled)[:, 1]

def create_interactive_prospect_plot(df):
    fig = px.scatter(
        df,
        x='potential_deal_size',
        y='time_on_site',
        color='prospect_probability',
        size='website_visits',
        hover_data=['company', 'industry'],
        title='Prospect Potential Visualization',
        labels={
            'potential_deal_size': 'Potential Deal Size ($)',
            'time_on_site': 'Time on Site (minutes)',
            'prospect_probability': 'Prospect Probability'
        },
        color_continuous_scale='viridis'
    )

    fig.update_layout(
        height=600,
        width=1000,
        template='plotly_white',
        coloraxis_colorbar=dict(title='Prospect Probability')
    )

    pio.write_html(fig, file='prospect_potential_visualization.html')

    fig.show()

    return fig

interactive_plot = create_interactive_prospect_plot(df)

print("\nProspect Probability Statistics:")
print(df['prospect_probability'].describe())

industry_potential = df.groupby('industry')['prospect_probability'].mean().sort_values(ascending=False)
print("\nAverage Prospect Probability by Industry:")
print(industry_potential)


Prospect Probability Statistics:
count    200.000000
mean       0.338129
std        0.289003
min        0.000000
25%        0.097500
50%        0.220000
75%        0.660000
max        0.940000
Name: prospect_probability, dtype: float64

Average Prospect Probability by Industry:
industry
Technology       0.404655
Healthcare       0.377513
Manufacturing    0.344553
Retail           0.343197
Logistics        0.338604
Finance          0.240603
Name: prospect_probability, dtype: float64


In [16]:
import pandas as pd
import numpy as np
import random
from faker import Faker
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from sklearn.preprocessing import MinMaxScaler

# Set up Faker for realistic data generation
fake = Faker()

# Custom Lead Generation Function
def generate_targeted_leads(num_leads=500):
    leads = []
    target_industries = ['Engineering', 'Manufacturing', 'Technology']
    contract_types = ['Options', 'Forwards', 'Both']

    for _ in range(num_leads):
        # Randomly select industry from target industries
        industry = random.choice(target_industries)

        # Generate realistic FX exposure
        fx_exposure = random.uniform(3000000, 40000000)

        # Select contract type
        contract_type = random.choice(contract_types)

        lead = {
            'lead_id': fake.uuid4(),
            'company_name': fake.company(),
            'industry': industry,
            'country': fake.country(),
            'annual_fx_exposure': fx_exposure,
            'contract_type': contract_type,

            # Additional contextual features
            'company_size': random.choice(['Small', 'Medium', 'Large']),
            'global_operations': random.choice([True, False]),
            'previous_fx_experience': random.uniform(0, 10),

            # Simulated interaction metrics
            'website_visits': random.randint(1, 10),
            'email_interactions': random.randint(0, 5),
            'content_downloads': random.randint(0, 3)
        }
        leads.append(lead)

    return pd.DataFrame(leads)

# Lead Scoring Algorithm
def calculate_lead_score(lead):
    # Base score components
    score = 0

    # Industry Scoring
    industry_score = {
        'Engineering': 25,
        'Manufacturing': 25,
        'Technology': 25
    }
    score += industry_score.get(lead['industry'], 0)

    # FX Exposure Scoring
    if 3000000 <= lead['annual_fx_exposure'] <= 40000000:
        # Normalize score based on exposure
        exposure_score = min(
            25,
            25 * (lead['annual_fx_exposure'] - 3000000) / (40000000 - 3000000)
        )
        score += exposure_score

    # Contract Type Scoring
    contract_score = {
        'Options': 20,
        'Forwards': 20,
        'Both': 25
    }
    score += contract_score.get(lead['contract_type'], 0)

    # Additional Contextual Scoring
    if lead['global_operations']:
        score += 10

    if lead['previous_fx_experience'] > 5:
        score += 10

    # Interaction Metrics Bonus
    interaction_score = min(
        15,
        lead['website_visits'] * 2 +
        lead['email_interactions'] * 3 +
        lead['content_downloads'] * 4
    )
    score += interaction_score

    # Ensure score is between 0-100
    return min(max(score, 0), 100)

# Generate Leads Dataset
leads_df = generate_targeted_leads()

# Apply Lead Scoring
leads_df['lead_score'] = leads_df.apply(calculate_lead_score, axis=1)

# Interactive Visualization
def create_lead_scoring_visualization(df):
    # Create scatter plot
    fig = px.scatter(
        df,
        x='annual_fx_exposure',
        y='lead_score',
        color='industry',
        size='lead_score',
        hover_data=[
            'company_name',
            'contract_type',
            'global_operations'
        ],
        title='Lead Scoring Analysis',
        labels={
            'annual_fx_exposure': 'Annual FX Exposure ($)',
            'lead_score': 'Lead Score',
            'industry': 'Industry'
        }
    )

    # Customize layout
    fig.update_layout(
        height=700,
        width=1200,
        template='plotly_white'
    )

    # Save interactive HTML
    pio.write_html(fig, file='lead_scoring_visualization.html')

    # Show the plot
    fig.show()

    return fig

# Generate Visualization
visualization = create_lead_scoring_visualization(leads_df)

# Detailed Analysis
def lead_scoring_analysis(df):
    # Summary Statistics
    print("\nLead Scoring Summary:")
    print(df['lead_score'].describe())

    # Top Leads
    top_leads = df[df['lead_score'] > 80].sort_values('lead_score', ascending=False)
    print("\nTop Leads (Score > 80):")
    print(top_leads[['company_name', 'industry', 'annual_fx_exposure', 'lead_score']].head())

    # Industry Distribution
    industry_distribution = df.groupby('industry')['lead_score'].agg(['mean', 'count'])
    print("\nLead Score by Industry:")
    print(industry_distribution)

    # Contract Type Analysis
    contract_analysis = df.groupby('contract_type')['lead_score'].agg(['mean', 'count'])
    print("\nLead Score by Contract Type:")
    print(contract_analysis)

    return top_leads

# Perform Analysis
top_prospects = lead_scoring_analysis(leads_df)

# Export Datasets
leads_df.to_csv('targeted_leads_with_scoring.csv', index=False)
top_prospects.to_csv('top_prospects.csv', index=False)

print("\nAnalysis complete. Check the generated files and visualization.")


Lead Scoring Summary:
count    500.000000
mean      82.847455
std       10.097254
min       55.366320
25%       75.856483
50%       82.466334
75%       90.917149
max      100.000000
Name: lead_score, dtype: float64

Top Leads (Score > 80):
                 company_name       industry  annual_fx_exposure  lead_score
101           Bartlett-Patton  Manufacturing        2.699037e+07       100.0
415                Lee-Hudson  Manufacturing        3.792406e+07       100.0
461  Blair, Wang and Johnston     Technology        3.531966e+07       100.0
378               Andrews LLC    Engineering        3.676225e+07       100.0
65                 Martin Ltd    Engineering        3.556039e+07       100.0

Lead Score by Industry:
                    mean  count
industry                       
Engineering    82.768267    166
Manufacturing  82.909091    160
Technology     82.866326    174

Lead Score by Contract Type:
                    mean  count
contract_type                  
Both           85.